In [1]:
from src.datasets import BasePostsDataset, BaseFactCheckDataset

tasks_path = "data/complete_data/tasks.json"
posts_path = "data/complete_data/posts.csv"
fact_checks_path = "data/complete_data/fact_checks.csv"
gs_path = "data/complete_data/pairs.csv"

posts = BasePostsDataset(posts_path, tasks_path, task_name="monolingual", gs_path=gs_path, lang="spa")
fact_checks = BaseFactCheckDataset(fact_checks_path, tasks_path, task_name="monolingual", lang="spa")

In [2]:
df_fc = fact_checks.df
df_posts_train = posts.df_train

In [3]:
df_fc.head()

,claim,instances,title
fact_check_id,,,
56,"""(A Pere Aragonés) ustedes eliminaron la conse...",[https://www.newtral.es/fact-check-medio-ambie...,El Departamento de Medio Ambiente de Cataluña ...
58,"""(Andalucía tiene) un 36% de paro juvenil de m...",[https://www.newtral.es/paro-juvenil-andalucia...,Andalucía no dobla al resto de España en paro ...
59,"""(Biden) destruirá la protección para las enfe...",[https://www.telemundo.com/noticias/noticias-t...,"No, Biden no eliminaría la cobertura de salud ..."
64,"""(El PP) vincula su oposición a renovar el CGP...",[https://www.newtral.es/factcheck-abalos-pp-re...,El PP no ha vinculado su oposición a renovar e...
65,"""(En Euskadi) llevamos 40 años de victorias el...",[https://www.newtral.es/el-bloque-de-izquierda...,El bloque de izquierda no lleva 40 años de vic...


In [4]:
df_posts_train.head()

,ocr,verdicts,text,fb,tw,ig,gs
post_id,,,,,,,
4,"""Bienaventurados los perseguidos por mi causa ...",,,1,0,0,[80729]
6,"""Coman lo que quieran en Semana Santa, el sacr...",False information,,1,0,0,[50769]
7,"""DISCURSO DE PEDRO CASTILLO, ESTUBO BASADO EN ...",False information,,1,0,0,[56968]
8,"""Debemos ser Solidarios con los que menos tien...",Partly false information,,1,0,0,[148668]
11,"""EL AGUINALDO DE 2 MILLONES NO ES SOLO PARA NO...",False information,,1,0,0,"[3332, 51854]"


In [5]:
df_fc["full_text"] = df_fc["title"] + " " + df_fc["claim"]
df_posts_train["full_text"] = df_posts_train["ocr"] + " " + df_posts_train["text"]

df_fc["full_text"] = df_fc["full_text"].str.lower()
df_posts_train["full_text"] = df_posts_train["full_text"].str.lower()

In [6]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
emb_post = model.encode(df_posts_train["full_text"].values)
emb_fc = model.encode(df_fc["full_text"].values)

/home/bsc/bsc830651/.conda/envs/factcheck/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [18]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

emb_post_sparse = sparse.csr_matrix(emb_post[0])
emb_fc_sparse = sparse.csr_matrix(emb_fc)

similarity = cosine_similarity(emb_post_sparse, emb_fc_sparse, dense_output=True)

# Sort in descending order
st_similarity = np.sort(similarity, axis=1)[:, ::-1]
idx_sim = np.argsort(similarity, axis=1)[:, ::-1]
print(st_similarity[:5],"\n", idx_sim[:5])

[[ 0.63616437  0.59415835  0.5922553  ... -0.08014088 -0.09203598
  -0.10629686]] 
 [[12614  7571   610 ...  3268  9068  2586]]


In [16]:
df_posts_train[df_posts_train["gs"].notnull()]

,ocr,verdicts,text,fb,tw,ig,gs,full_text
post_id,,,,,,,,
4,"""Bienaventurados los perseguidos por mi causa ...",,,1,0,0,[80729],"""bienaventurados los perseguidos por mi causa ..."
6,"""Coman lo que quieran en Semana Santa, el sacr...",False information,,1,0,0,[50769],"""coman lo que quieran en semana santa, el sacr..."
7,"""DISCURSO DE PEDRO CASTILLO, ESTUBO BASADO EN ...",False information,,1,0,0,[56968],"""discurso de pedro castillo, estubo basado en ..."
8,"""Debemos ser Solidarios con los que menos tien...",Partly false information,,1,0,0,[148668],"""debemos ser solidarios con los que menos tien..."
11,"""EL AGUINALDO DE 2 MILLONES NO ES SOLO PARA NO...",False information,,1,0,0,"[3332, 51854]","""el aguinaldo de 2 millones no es solo para no..."
...,...,...,...,...,...,...,...,...
22499,25% 20% 15% 10% 5% 2.3% Rafael Lopez Aliaga EN...,False information,Última encuesta de la PUCP!,1,0,0,[29594],25% 20% 15% 10% 5% 2.3% rafael lopez aliaga en...
22501,ÚLTIMO MINUTO COLOMBIA NO RECONOCERÁ A JOE BID...,Altered photo,Última hora...,1,0,0,[141012],último minuto colombia no reconocerá a joe bid...
22503,,False information,"Último adiós, South African Airways Noticia: [...",1,0,0,[83516],"último adiós, south african airways noticia: ..."


In [20]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def get_k_similar_posts(post, fc_pool, k=5):
    emb_post_sparse = sparse.csr_matrix(post)
    emb_fc_sparse = sparse.csr_matrix(fc_pool)

    similarity = cosine_similarity(emb_post_sparse, emb_fc_sparse, dense_output=True)
    similarity = similarity[0]
    # Sort in descending order
    st_similarity = np.sort(similarity)[::-1]
    idx_sim = np.argsort(similarity)[::-1]
    print(st_similarity[:k],"\n", idx_sim[:k])
    return idx_sim[:k], st_similarity[:k]

def display_similar_posts(post_id, emb_post, emb_fc):
    idx_to_pos = {idx: pos for pos, idx in enumerate(df_posts_train.index)}
    idx_sim, similarity = get_k_similar_posts(emb_post[idx_to_pos[post_id]], emb_fc, k=5)

    # Show the full text of the post
    print(df_posts_train.loc[post_id, ].to_markdown())
    print("="*100)
    print("\n\n")
    for i, idx in enumerate(idx_sim):
        print(f"Similarity: {similarity[i]}")
        print(df_fc.iloc[idx].to_markdown())
        print("\n\n")


display_similar_posts(11, emb_post, emb_fc)


[0.6850206 0.655922  0.6496488 0.6137628 0.6074511] 
 [ 4178 13328   218  9894  9585]
|           | 11                                                                                                                                                                                                                                |
|:----------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| ocr       | "EL AGUINALDO DE 2 MILLONES NO ES SOLO PARA NOSOTROS ES PARA NUESTROS ASESORES ELLOS TAMBIÉN TIENEN FAMILIA. ASI QUE LE PIDO A LA CAN DERECHA QUE DEJE DE MENTIR" CHILE En el listado de las 100 personas más influyentes de 2021 |
| verdicts  | False information                                                                                                                                                             

In [34]:
print(df_posts_train[["full_text", "gs"]].iloc[0].to_markdown())

|           | 4                                                                                                                                                                                                                                                                                         |
|:----------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| full_text | "bienaventurados los perseguidos por mi causa " ellos alcanzarán el cielo. son religiosas apresadas por defender a los niños presos en jaulas, hijos de los emigrantes detenidos en los eeuu, para deportarlos. ayuden, conpartiendo este hecho, ya que ningun periódico lo da a conocer. |
| gs        | [80729]                                                                                     

In [38]:
print(df_fc.loc[80729, :].to_markdown())

|           | 80729                                                                                                                                                                                                                                              |
|:----------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| claim     | La foto muestra a monjas detenidas por participar de una manifestación contra la detención de niños migrantes en Estados Unidos                                                                                                                    |
| instances | ['https://factual.afp.com/la-monja-que-aparece-esposada-fue-arrestada-en-estados-unidos-por-protestar-contra-la-detencion-de#7e3e92910284094a31d70ab09eaae030']                                                  

In [36]:
print(df_fc.iloc[idx_sim[0, :20]].to_markdown())

|   fact_check_id | claim                                                                                                                                                                                                                                                      | instances                                                                                                                                                                                       | title                                                                                                                                                                                                    | full_text                                                                                                                                                                                                                                                                                                                                 

In [10]:
similarity.sort(axis=1)



In [25]:
np.dot(emb_post[0], emb_fc[0]) / (np.linalg.norm(emb_post[0]) * np.linalg.norm(emb_fc[0]))

np.float32(0.24843584)

In [27]:
np.dot(emb_post[0], emb_fc[1]) / (np.linalg.norm(emb_post[0]) * np.linalg.norm(emb_fc[1]))

np.float32(0.20492055)

In [28]:
np.dot(emb_post[1], emb_fc[0]) / (np.linalg.norm(emb_post[1]) * np.linalg.norm(emb_fc[0]))

np.float32(0.3132233)